In [17]:
# !pip install -r "E:\Tasks\Webinar\agentdemo\requirements.txt"

In [18]:
from datetime import datetime
from typing_extensions import Annotated
import autogen
import os
from dotenv import load_dotenv

load_dotenv()

True

In [19]:
def get_config():
    """Load Azure API configuration from environment variables."""
    return [
        {
            "model": os.getenv("AZURE_API_DEPLOYMENT"),
            "api_type": "azure",
            "api_key": os.getenv("AZURE_API_KEY"),
            "base_url": os.getenv("AZURE_API_ENDPOINT"),
            "api_version": os.getenv("AZURE_API_VERSION"),
        }
    ]

def get_llm_config():
    """Return LLM configuration."""
    return {
        "cache_seed": 42,
        "temperature": 0,
        "config_list": get_config(),
        "timeout": 120,
    }

In [20]:
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
    code_execution_config=False,
)

In [21]:
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=get_llm_config(),
    system_message="""Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
""",
)

In [22]:
scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config=get_llm_config(),
    system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts printed. You don't write code.""",
)

In [23]:
planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a scientist who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.
""",
    llm_config=get_llm_config(),
)

In [24]:
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

In [25]:
critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.",
    llm_config=get_llm_config(),
)

In [26]:
default_path = "E:/Personal/Kishore/Career/AzureDeveloperCommunity/CommunityEvents/Feb-15-MSUS/AIAgentCode/backend_dir/"

@user_proxy.register_for_execution()
@engineer.register_for_llm(description="List files in choosen directory.")
def list_dir(directory: Annotated[str, "Directory to check."]):
    files = os.listdir(default_path + directory)
    return 0, files

In [27]:
@user_proxy.register_for_execution()
@engineer.register_for_llm(description="Check the contents of a chosen file.")
def see_file(filename: Annotated[str, "Name and path of file to check."]):
    with open(default_path + filename, "r") as file:
        lines = file.readlines()
    formatted_lines = [f"{i+1}:{line}" for i, line in enumerate(lines)]
    file_contents = "".join(formatted_lines)

    return 0, file_contents

In [28]:
@user_proxy.register_for_execution()
@engineer.register_for_llm(description="Replace old piece of code with new one. Proper indentation is important.")
def modify_code(
    filename: Annotated[str, "Name and path of file to change."],
    start_line: Annotated[int, "Start line number to replace with new code."],
    end_line: Annotated[int, "End line number to replace with new code."],
    new_code: Annotated[str, "New piece of code to replace old code with. Remember about providing indents."],
):
    with open(default_path + filename, "r+") as file:
        file_contents = file.readlines()
        file_contents[start_line - 1: end_line] = [new_code + "\n"]
        file.seek(0)
        file.truncate()
        file.write("".join(file_contents))
    return 0, "Code modified"

In [29]:
@user_proxy.register_for_execution()
@engineer.register_for_llm(description="Create a new file with code.")
def create_file_with_code(
    filename: Annotated[str, "Name and path of file to create."], code: Annotated[str, "Code to write in the file."]
):
    with open(default_path + filename, "w") as file:
        file.write(code)
    return 0, "File created successfully"

In [30]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, scientist, planner,
            executor, critic], messages=[], max_round=50
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=get_llm_config(), code_execution_config={
    "work_dir": "coding",
    "use_docker": False,
},)

In [31]:
response = user_proxy.initiate_chat(
    manager,
    message="""
Ask for questions and do the instructions.
""",
)

Admin (to chat_manager):


Ask for questions and do the instructions.


--------------------------------------------------------------------------------

Next speaker: Admin

Admin (to chat_manager):

write a research papaer on agentic AI

--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

### Initial Plan: Writing a Research Paper on Agentic AI

The goal is to collaboratively write a research paper on "Agentic AI," which refers to artificial intelligence systems that exhibit autonomous decision-making and goal-directed behavior. The process will involve both a **scientist** (to conduct research, analyze concepts, and write the theoretical framework) and an **engineer** (to implement and demonstrate practical examples or simulations of agentic AI). Below is the step-by-step plan:

---

### Step 1: Define the Scope and Structure of the Paper (Scientist)
- **Objective**: The scientist will define the scope 